In [1]:
import iris
import pygrib

pygribでGRIBを読み込む

In [2]:
grbs = pygrib.open('/Users/k-ikegami/Desktop/GRIB/weather/Z__C_RJTD_20170120090000_MSM_GPV_Rjp_L-pall_FH00-15_grib2.bin')

iris cubeに必要な要素をリストアップ

- standard name (like "air temperature")
- data array (numpy)
- coordinate dimension 0
- coordinate dimension 1
- coordinate dimension 2

additionally 
- a data array of length 1 representing the time
- a long name of place name and no unit
- an auxiliary coordinate “factory”, which can derive its own mapping

iris.cube.Cube, iris.cube.CubeMetadata, iris.coords.DimCoord,  iris.coords.Coordあたりを使う

In [3]:
# 特定の物理量を取ってくる
grb = grbs.select(forecastTime=0)[0]
print(grb)

1:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201701200900


In [4]:
# そのlon, latを取ってくる
# pygrbでは緯度経度がグリッドに対応した形、つまりそれぞれが二次元配列で入っているので、irisに使うための要素のみを　取り出します
lats, lons = grb.latlons()
lat = lats[:, 0]
lon = lons[0, :]

In [5]:
# grib dataに入っている気象データ
values = grb.values

In [14]:
# cubeにする
# time dimension無しの２次元データ
from iris.coords import DimCoord
from iris.cube import Cube
import numpy as np

latitude = DimCoord(lat,standard_name='latitude',units='degrees')
longitude = DimCoord(lon,standard_name='longitude',units='degrees')

In [20]:
cube = Cube(values,dim_coords_and_dims=[(latitude,0),(longitude,1)])

In [8]:
print(grb.name)
print(grb.units)

Geopotential Height
gpm


In [9]:
type(grb.units)

str

In [10]:
# name,unitやらをpygribから取ってこれるか？

# name = stndard name
name = grb.name
# unit
unit = grb.units

# cubeに追加
latitude = DimCoord(lat,standard_name='latitude',units='degrees')
longitude = DimCoord(lon,standard_name='longitude',units='degrees')
cube = Cube(values,　standard_name = name, units = unit, dim_coords_and_dims=[(latitude, 0),(longitude, 1)])
print(cube)

SyntaxError: invalid character in identifier (<ipython-input-10-32c55af34e47>, line 11)

In [21]:
# time dimensionとか高さとか取り込んでみる
# まだやれてない

irisのconstraint objectはここで定義

http://scitools.org.uk/iris/docs/latest/iris/iris.html#iris.Constraint

In [28]:
# 分割してみましょう
# 20 < lat < 40, 130 < lon < 140 
slicecube = cube.extract(iris.Constraint(coord_values={'latitude':lambda cell: 20 < cell < 40, 'longitude':lambda cell: 130 < cell < 140},))
print(slicecube)

unknown / (unknown)                 (latitude: 177; longitude: 79)
     Dimension coordinates:
          latitude                           x               -
          longitude                          -               x


ということで数値の切り出しはどうにかなる。（他のcoordinateは必要に応じて対応する。）

なので、grib dataでの書き出しもやっておこう。→pygribにそもそもそんな機能はついてない。

irisで他のフォーマットならなんとかなる。

In [34]:
iris.save(slicecube, '/Users/k-ikegami/Desktop/GRIB/weather/slice.nc')

/Users/k-ikegami/anaconda/lib/python3.5/site-packages/iris/fileformats/netcdf.py:2281: IrisDeprecation: NetCDF default saving behaviour currently assigns the outermost dimensions to unlimited. This behaviour is to be deprecated, in favour of no automatic assignment. To switch to the new behaviour, set iris.FUTURE.netcdf_no_unlimited to True.
  warn_deprecated(msg)


In [35]:
cubes = iris.load("slice.nc")
print(cubes)

0: unknown / (1)                       (latitude: 177; longitude: 79)


/Users/k-ikegami/anaconda/lib/python3.5/site-packages/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


In [41]:
cubes[0][0,2]

<iris 'Cube' of unknown / (1) (scalar cube)>

こっから下はirisの動作確認

In [13]:
filename = iris.sample_data_path('rotated_pole.nc')
cube = iris.load_cube(filename)
cube.coords()

/Users/k-ikegami/anaconda/lib/python3.5/site-packages/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


[DimCoord(array([-22.48999977, -20.28999901, -18.09000015, -15.88999939,
        -13.68999958, -11.48999977,  -9.28999996,  -7.09000015,
         -4.88999939,  -2.69000053,  -0.48999977,   1.71000099,
          3.90999985,   6.11000061,   8.30999947,  10.51000023,
         12.71000099,  14.91000175,  17.1099987 ,  19.30999947,
         21.51000023,  23.71000099], dtype=float32), standard_name='grid_latitude', units=Unit('degrees'), var_name='grid_latitude', coord_system=RotatedGeogCS(37.5, 177.5, ellipsoid=GeogCS(6371229.0))),
 DimCoord(array([ 313.01998901,  315.22000122,  317.41998291,  319.61999512,
         321.81997681,  324.01998901,  326.22000122,  328.41998291,
         330.61999512,  332.81997681,  335.01998901,  337.22000122,
         339.41998291,  341.61999512,  343.81997681,  346.01998901,
         348.22000122,  350.41998291,  352.61999512,  354.81997681,
         357.01998901,  359.22000122,  361.41998291,  363.61999512,
         365.81997681,  368.01998901,  370.2200012

In [14]:
for coord in cube.coords():
    print(coord.name())

grid_latitude
grid_longitude
forecast_period
forecast_reference_time
time


In [16]:
coord = cube.coord('grid_latitude')
coord

DimCoord(array([-22.48999977, -20.28999901, -18.09000015, -15.88999939,
       -13.68999958, -11.48999977,  -9.28999996,  -7.09000015,
        -4.88999939,  -2.69000053,  -0.48999977,   1.71000099,
         3.90999985,   6.11000061,   8.30999947,  10.51000023,
        12.71000099,  14.91000175,  17.1099987 ,  19.30999947,
        21.51000023,  23.71000099], dtype=float32), standard_name='grid_latitude', units=Unit('degrees'), var_name='grid_latitude', coord_system=RotatedGeogCS(37.5, 177.5, ellipsoid=GeogCS(6371229.0)))

In [18]:
# 実際にcubeを作ってみる
from iris.coords import DimCoord
from iris.cube import Cube
import numpy as np

latitude = DimCoord(np.linspace(-90, 90, 4),standard_name='latitude',units='degrees')

longitude = DimCoord(np.linspace(45, 360, 8),standard_name='longitude',units='degrees')

cube = Cube(np.zeros((4, 8), np.float32),dim_coords_and_dims=[(latitude, 0),(longitude, 1)])

In [12]:
filename = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(filename)
iris.fileformats.grib.save_grib2(cubes)

AttributeError: module 'iris.fileformats' has no attribute 'grib'